<a href="https://colab.research.google.com/github/dmitryglhf/jupyter-projects/blob/main/lab_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Data

In [ ]:
import numpy as np
import pandas as pd

#from fedot.api.main import Fedot

In [ ]:
#df = pd.read_csv('our data....')

### QR-algorithm


In [ ]:
def householder(a):
    v = a.copy()
    v[0] += np.sign(v[0]) * np.linalg.norm(a)
    H = np.eye(len(a)) - 2 * np.outer(v, v) / np.dot(v, v)  # getting Householder matrix
    return H


def get_qr(A):
    n = A.shape[0]
    Q = np.eye(n)
    R = A.copy()

    for j in range(n):
        a = R[j:, j]  # getting vector
        H = householder(a)  # getting Householder matrix

        R[j:, j:] = H @ R[j:, j:]  # change R
        Q[:, j:] = Q[:, j:] @ H.T  # change Q

    return Q, R


def eig(A, e=1e-10, max_iter = 1000):
    An = A.copy() # for eig values
    Qk = np.eye(A.shape[0]) # for eig vectors
    stop_check = np.sqrt(np.sum(np.tril(An, -1) ** 2)) # under diag values
    iterations = 0

    while stop_check > e and iterations < max_iter:
        Q, R = get_qr(An)
        An = R @ Q
        Qk = Qk @ Q
        stop_check = np.sqrt(np.sum(np.tril(An, -1) ** 2))
        iterations += 1

    eig_values = np.diag(An)


    # Return eigen values and eigen vectors
    '''
    1. Создать словарь ключ: собственное число, значение: собственный вектор
    2. Создать отсортировать по ключу
    3. Восстановить матрицу собственных векторов с учетом сортировки
    '''
    return eig_values, Qk, An

### SVD

In [ ]:
def svd(A):
  # V^T matrix
  temp_V = A.T @ A
  eig_values, Qk = eig(temp_V)
  V_T = Qk.T

  # U matrix
  temp_U = A @ A.T
  _, Qk = eig(temp_U)
  U = Qk

  # Sigma matrix
  sigma = [[0] * len(eig_values) for _ in range(len(eig_values))]
  for i in range(len(eig_values)):
        sigma[i][i] = np.sqrt(eig_values[i])
  # Result
  return U, sigma, V_T

### Test

In [ ]:

'''
eigen values for matrix A:
l1 = -2
l2 = 2
l3 = 4

eigen vectors for matrix A:
v1 = (0, 1, -2)
v2 = (2, -1, 0)
v3 = (9, -2, 1)

original: http://mathprofi.ru/sobstvennye_znachenija_i_sobstvennye_vektory.html
'''

A = np.array([[5, 6, 3],
                      [-1, 0, 1],
                      [1, 2, -1]], dtype = float)
A=A.T@A
print(A)
eig_values, Qk, An = eig(A)

n = A.shape[0]

#for i in range(n):
  #v = Qk[:, i]  #[x[i] for x in Qk]
  #print(np.linalg.norm((A - eig_values[i] @ np.eye(n))@v))


for i in range(n):
    v = Qk[:, i]  # Извлечение i-го собственного вектора
    # Вычисление нормы выражения (A - eig_value * I) @ v
    norm_value = np.linalg.norm((A - eig_values[i] * np.eye(n)) @ v)

    print(f"Норма для собственного значения {eig_values[i]}: {norm_value}")

#for eig in eig_values:

  #for i in range(A.shape[0]):
    #v = [x[i] for x in Qk]
    #left_side = A @ v
    #right_side = eig * v
    #print(left_side - right_side)

print('Matrix:')
print(Qk)
#print('\nEig values:')
#print(eig_values)


[[27. 32. 13.]
 [32. 40. 16.]
 [13. 16. 11.]]
Норма для собственного значения 0.845548088315283: 72.16189950153716
Норма для собственного значения 4.147004321832275: 7.13149831455435e-11
Норма для собственного значения 73.00744758985249: 72.1618995015372
Matrix:
[[ 0.60030706 -0.21284918  0.77092585]
 [ 0.73491404 -0.23343825 -0.63671653]
 [ 0.31548817  0.94878965  0.01629122]]


### Комментарии

Делаем так: нам нужно найти разложение X вида $U*Сигма*V_T$:
- находим собственные значения для матрицы A*A_T и собств вектора -> определим U и Сигму из этого

Cейчас (до вс) нужно научиться сортировать в сигме синг значения по убыванию и менять соответсвующие столбцы в U.
Что делать с матрицей V я подумаю и скажу в вс